In [1]:
import pandas as pd
import numpy as np
from scipy.stats import binned_statistic_2d
%pylab inline
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
from itertools import product

Populating the interactive namespace from numpy and matplotlib


C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
dat= pd.read_csv('dat_c6w6.csv', sep=';')
dat['tim1'] = dat['tim1'].astype(datetime64)
dat.set_index('Unnamed: 0', inplace = True) #'Unnamed: 0'


In [3]:
dat

,zone,tim1,hour,day,week_day,week,month,year,val,sin1,...,payment_type_4,payment_type_5,payment_type_6,RatecodeID_1,RatecodeID_2,RatecodeID_3,RatecodeID_4,RatecodeID_5,RatecodeID_6,time_len
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,1075,2015-06-01 00:00:00,0,1,0,23,6,2015,40,0.037391,...,0.0,0.0,0.0,0.975,0.025,0.0,0.0,0.0,0.0,661375000000
1,1075,2015-06-01 01:00:00,1,1,0,23,6,2015,17,0.074730,...,0.0,0.0,0.0,1.000,0.000,0.0,0.0,0.0,0.0,978411764705
2,1075,2015-06-01 02:00:00,2,1,0,23,6,2015,5,0.111964,...,0.0,0.0,0.0,1.000,0.000,0.0,0.0,0.0,0.0,982000000000
3,1075,2015-06-01 03:00:00,3,1,0,23,6,2015,5,0.149042,...,0.0,0.0,0.0,0.800,0.200,0.0,0.0,0.0,0.0,1041200000000
4,1075,2015-06-01 04:00:00,4,1,0,23,6,2015,8,0.185912,...,0.0,0.0,0.0,1.000,0.000,0.0,0.0,0.0,0.0,923000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969403,2168,2016-06-30 19:00:00,19,30,3,26,6,2016,1,-0.294755,...,0.0,0.0,0.0,0.000,1.000,0.0,0.0,0.0,0.0,3913000000000
969404,2168,2016-06-30 20:00:00,20,30,3,26,6,2016,0,-0.330279,...,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0
969405,2168,2016-06-30 21:00:00,21,30,3,26,6,2016,1,-0.365341,...,0.0,0.0,0.0,0.000,0.000,0.0,0.0,1.0,0.0,20000000000


In [4]:
%%time
train_start = dat['tim1'][8*168]
train_stop = "2016-05-01 00:00:00"
test_stop = "2016-05-31 17:00:00"
pred_start = "2016-05-31 23:00:00"
pred_stop = "2016-06-30 17:00:00"
dat_train = dat[(dat.tim1 >= train_start) & (dat.tim1 <= train_stop)]
dat_test = dat[(dat.tim1 >= train_stop) & (dat.tim1 <= pred_start)]
dat_train_pred = dat[(dat.tim1 >= train_start) & (dat.tim1 <= pred_start)]
dat_pred = dat[(dat.tim1 >= pred_start) & (dat.tim1 <= pred_stop)]

Wall time: 7.49 s


In [5]:
dat.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            969398, 969399, 969400, 969401, 969402, 969403, 969404, 969405,
            969406, 969407],
           dtype='int64', name='Unnamed: 0', length=969408)

In [6]:
from catboost import CatBoostRegressor

In [7]:
%%time
# Создаем мега цикл, который посчитает качество модели и остатки в мае, только попробуем другую модель.
xgb_params = {'learning_rate': [0.1], 'iterations': [1000], 'depth':[10]}
params = list(product(xgb_params['learning_rate'], xgb_params['iterations'], xgb_params['depth']))
X_test = dat_test.copy()
X_test.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_test.drop(X_test.loc[:, 'sin1':'cos50'].columns, inplace=True, axis=1)
X_train = dat_train.copy()
X_train.drop(['tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_train.drop(X_train.loc[:, 'sin1':'cos50'].columns, inplace=True, axis=1)

best_xgb_cv = None
#best_xgb_score = float('inf')
Q_xgb = 40*102*739*6.
best_xgb_params = None
for j in range(len(params)):
    Q_xgb_1 = 0
    param = params[j]
    xgb_cv = []
    xgb_model = CatBoostRegressor(learning_rate=param[0], iterations=param[1], depth=param[2], loss_function='RMSE',
                                    #boosting_type='Plain', loss_function='MultiClass'
            one_hot_max_size=20, )
    for i in range(1,7):
        Q_may = 0. # ошибка в мае
        Y_test = dat_test[str(i) + '_ans']
        Y_train = dat_train[str(i) + '_ans']
        
        
        
        #Считаем ошибку по линейной регрессии
        stud = xgb_model.fit(X_train, Y_train)
    
    
        Y_pred = stud.predict(X_test)
        Y_pred[Y_pred < 0] = 0  # All low values set to 0
        Y_score = stud.score(X_test, Y_test)
        Q_may += sum(abs(Y_test-Y_pred))
        Q_xgb_1 += Q_may
        print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_may))
        print(' ')
    
    print ('Ошибка данной модели Catboost: '+str(1/(102*739*6.)*Q_xgb_1)+', при параметрах: '+ str(param))
    if Q_xgb_1 < Q_xgb:
        Q_xgb = Q_xgb_1
        best_xgb_cv = xgb_cv
        best_xgb_params = param
print('Лучшие параметры:' +str(best_xgb_params) + ' , ошибка:' + str(1/(102*739*6.)*Q_xgb))

0:	learn: 226.7631716	total: 833ms	remaining: 13m 52s
1:	learn: 205.8940735	total: 1.54s	remaining: 12m 47s
2:	learn: 187.3045213	total: 2.24s	remaining: 12m 24s
3:	learn: 170.4367063	total: 2.94s	remaining: 12m 13s
4:	learn: 155.5458804	total: 3.7s	remaining: 12m 15s
5:	learn: 142.2450586	total: 4.43s	remaining: 12m 14s
6:	learn: 130.3549704	total: 5.17s	remaining: 12m 13s
7:	learn: 119.6065866	total: 5.92s	remaining: 12m 13s
8:	learn: 109.9681848	total: 6.64s	remaining: 12m 11s
9:	learn: 101.3630321	total: 7.37s	remaining: 12m 9s
10:	learn: 93.7013833	total: 8.11s	remaining: 12m 9s
11:	learn: 86.8114717	total: 8.87s	remaining: 12m 10s
12:	learn: 80.7070900	total: 9.62s	remaining: 12m 10s
13:	learn: 75.3449466	total: 10.3s	remaining: 12m 8s
14:	learn: 70.6746211	total: 11s	remaining: 12m 5s
15:	learn: 66.4867385	total: 11.8s	remaining: 12m 4s
16:	learn: 62.7747053	total: 12.5s	remaining: 12m 4s
17:	learn: 59.5287627	total: 13.3s	remaining: 12m 3s
18:	learn: 56.6694267	total: 14s	remai

151:	learn: 29.5216320	total: 1m 53s	remaining: 10m 35s
152:	learn: 29.4890201	total: 1m 54s	remaining: 10m 34s
153:	learn: 29.4537520	total: 1m 55s	remaining: 10m 33s
154:	learn: 29.4196210	total: 1m 56s	remaining: 10m 32s
155:	learn: 29.3877485	total: 1m 56s	remaining: 10m 32s
156:	learn: 29.3657542	total: 1m 57s	remaining: 10m 31s
157:	learn: 29.3295021	total: 1m 58s	remaining: 10m 30s
158:	learn: 29.3001583	total: 1m 59s	remaining: 10m 30s
159:	learn: 29.2684611	total: 1m 59s	remaining: 10m 29s
160:	learn: 29.2359220	total: 2m	remaining: 10m 28s
161:	learn: 29.2082544	total: 2m 1s	remaining: 10m 28s
162:	learn: 29.1873422	total: 2m 2s	remaining: 10m 27s
163:	learn: 29.1624839	total: 2m 2s	remaining: 10m 26s
164:	learn: 29.1389882	total: 2m 3s	remaining: 10m 26s
165:	learn: 29.1061346	total: 2m 4s	remaining: 10m 25s
166:	learn: 29.0710386	total: 2m 5s	remaining: 10m 24s
167:	learn: 29.0466765	total: 2m 5s	remaining: 10m 23s
168:	learn: 29.0183205	total: 2m 6s	remaining: 10m 23s
169:

301:	learn: 26.3677163	total: 3m 53s	remaining: 8m 58s
302:	learn: 26.3524136	total: 3m 53s	remaining: 8m 57s
303:	learn: 26.3430691	total: 3m 54s	remaining: 8m 57s
304:	learn: 26.3262286	total: 3m 55s	remaining: 8m 56s
305:	learn: 26.3078732	total: 3m 56s	remaining: 8m 55s
306:	learn: 26.2939320	total: 3m 56s	remaining: 8m 54s
307:	learn: 26.2754136	total: 3m 57s	remaining: 8m 54s
308:	learn: 26.2623562	total: 3m 58s	remaining: 8m 53s
309:	learn: 26.2511944	total: 3m 59s	remaining: 8m 52s
310:	learn: 26.2399545	total: 4m	remaining: 8m 51s
311:	learn: 26.2221774	total: 4m	remaining: 8m 51s
312:	learn: 26.2038072	total: 4m 1s	remaining: 8m 50s
313:	learn: 26.1905411	total: 4m 2s	remaining: 8m 49s
314:	learn: 26.1710960	total: 4m 3s	remaining: 8m 49s
315:	learn: 26.1551541	total: 4m 4s	remaining: 8m 48s
316:	learn: 26.1396299	total: 4m 4s	remaining: 8m 47s
317:	learn: 26.1257684	total: 4m 5s	remaining: 8m 46s
318:	learn: 26.1094915	total: 4m 6s	remaining: 8m 46s
319:	learn: 26.0960685	to

452:	learn: 24.4412319	total: 5m 51s	remaining: 7m 4s
453:	learn: 24.4334534	total: 5m 52s	remaining: 7m 4s
454:	learn: 24.4239837	total: 5m 53s	remaining: 7m 3s
455:	learn: 24.4143863	total: 5m 54s	remaining: 7m 2s
456:	learn: 24.4074364	total: 5m 54s	remaining: 7m 1s
457:	learn: 24.4012590	total: 5m 55s	remaining: 7m
458:	learn: 24.3873723	total: 5m 56s	remaining: 7m
459:	learn: 24.3718026	total: 5m 57s	remaining: 6m 59s
460:	learn: 24.3579118	total: 5m 58s	remaining: 6m 58s
461:	learn: 24.3458542	total: 5m 58s	remaining: 6m 58s
462:	learn: 24.3363050	total: 5m 59s	remaining: 6m 57s
463:	learn: 24.3292582	total: 6m	remaining: 6m 56s
464:	learn: 24.3194734	total: 6m 1s	remaining: 6m 55s
465:	learn: 24.3078364	total: 6m 2s	remaining: 6m 54s
466:	learn: 24.2945218	total: 6m 2s	remaining: 6m 54s
467:	learn: 24.2818901	total: 6m 3s	remaining: 6m 53s
468:	learn: 24.2715116	total: 6m 4s	remaining: 6m 52s
469:	learn: 24.2610409	total: 6m 5s	remaining: 6m 51s
470:	learn: 24.2539140	total: 6m 

602:	learn: 23.0527707	total: 7m 56s	remaining: 5m 13s
603:	learn: 23.0448512	total: 7m 57s	remaining: 5m 12s
604:	learn: 23.0338390	total: 7m 58s	remaining: 5m 12s
605:	learn: 23.0284482	total: 7m 59s	remaining: 5m 11s
606:	learn: 23.0226502	total: 7m 59s	remaining: 5m 10s
607:	learn: 23.0147425	total: 8m	remaining: 5m 9s
608:	learn: 23.0082214	total: 8m 1s	remaining: 5m 9s
609:	learn: 22.9962582	total: 8m 2s	remaining: 5m 8s
610:	learn: 22.9901231	total: 8m 3s	remaining: 5m 7s
611:	learn: 22.9791462	total: 8m 4s	remaining: 5m 6s
612:	learn: 22.9709493	total: 8m 5s	remaining: 5m 6s
613:	learn: 22.9640303	total: 8m 6s	remaining: 5m 5s
614:	learn: 22.9558210	total: 8m 7s	remaining: 5m 5s
615:	learn: 22.9495468	total: 8m 8s	remaining: 5m 4s
616:	learn: 22.9447582	total: 8m 9s	remaining: 5m 3s
617:	learn: 22.9375256	total: 8m 9s	remaining: 5m 2s
618:	learn: 22.9282573	total: 8m 10s	remaining: 5m 2s
619:	learn: 22.9198574	total: 8m 11s	remaining: 5m 1s
620:	learn: 22.9096815	total: 8m 12s	

...

381:	learn: 32.9996100	total: 5m 17s	remaining: 8m 33s
382:	learn: 32.9857437	total: 5m 17s	remaining: 8m 32s
383:	learn: 32.9698281	total: 5m 18s	remaining: 8m 31s
384:	learn: 32.9606163	total: 5m 19s	remaining: 8m 30s
385:	learn: 32.9406691	total: 5m 20s	remaining: 8m 29s
386:	learn: 32.9231354	total: 5m 21s	remaining: 8m 28s
387:	learn: 32.9073575	total: 5m 22s	remaining: 8m 27s
388:	learn: 32.8534451	total: 5m 22s	remaining: 8m 27s
389:	learn: 32.8397710	total: 5m 23s	remaining: 8m 26s
390:	learn: 32.8309455	total: 5m 24s	remaining: 8m 25s
391:	learn: 32.8064722	total: 5m 25s	remaining: 8m 24s
392:	learn: 32.7881252	total: 5m 26s	remaining: 8m 23s
393:	learn: 32.7582546	total: 5m 26s	remaining: 8m 22s
394:	learn: 32.7338430	total: 5m 27s	remaining: 8m 21s
395:	learn: 32.7221946	total: 5m 28s	remaining: 8m 21s
396:	learn: 32.7054360	total: 5m 29s	remaining: 8m 20s
397:	learn: 32.6862239	total: 5m 30s	remaining: 8m 19s
398:	learn: 32.6704556	total: 5m 30s	remaining: 8m 18s
399:	learn

532:	learn: 30.6562198	total: 7m 20s	remaining: 6m 25s
533:	learn: 30.6389060	total: 7m 21s	remaining: 6m 25s
534:	learn: 30.6299120	total: 7m 22s	remaining: 6m 24s
535:	learn: 30.6118515	total: 7m 23s	remaining: 6m 23s
536:	learn: 30.6004476	total: 7m 24s	remaining: 6m 22s
537:	learn: 30.5897260	total: 7m 25s	remaining: 6m 22s
538:	learn: 30.5811187	total: 7m 25s	remaining: 6m 21s
539:	learn: 30.5680535	total: 7m 26s	remaining: 6m 20s
540:	learn: 30.5583550	total: 7m 27s	remaining: 6m 19s
541:	learn: 30.5479992	total: 7m 28s	remaining: 6m 19s
542:	learn: 30.5389839	total: 7m 30s	remaining: 6m 18s
543:	learn: 30.5310227	total: 7m 31s	remaining: 6m 18s
544:	learn: 30.5210844	total: 7m 32s	remaining: 6m 17s
545:	learn: 30.5046431	total: 7m 33s	remaining: 6m 16s
546:	learn: 30.4911707	total: 7m 34s	remaining: 6m 16s
547:	learn: 30.4726311	total: 7m 35s	remaining: 6m 15s
548:	learn: 30.4633934	total: 7m 36s	remaining: 6m 14s
549:	learn: 30.4490405	total: 7m 37s	remaining: 6m 14s
550:	learn

683:	learn: 28.8939891	total: 9m 46s	remaining: 4m 31s
684:	learn: 28.8770977	total: 9m 48s	remaining: 4m 30s
685:	learn: 28.8693873	total: 9m 49s	remaining: 4m 29s
686:	learn: 28.8565939	total: 9m 50s	remaining: 4m 29s
687:	learn: 28.8386741	total: 9m 52s	remaining: 4m 28s
688:	learn: 28.8302815	total: 9m 53s	remaining: 4m 27s
689:	learn: 28.8214039	total: 9m 54s	remaining: 4m 26s
690:	learn: 28.8063998	total: 9m 55s	remaining: 4m 26s
691:	learn: 28.7917862	total: 9m 56s	remaining: 4m 25s
692:	learn: 28.7836792	total: 9m 57s	remaining: 4m 24s
693:	learn: 28.7751283	total: 9m 58s	remaining: 4m 23s
694:	learn: 28.7675010	total: 9m 59s	remaining: 4m 23s
695:	learn: 28.7552891	total: 10m	remaining: 4m 22s
696:	learn: 28.7419398	total: 10m 1s	remaining: 4m 21s
697:	learn: 28.7335327	total: 10m 2s	remaining: 4m 20s
698:	learn: 28.7190580	total: 10m 3s	remaining: 4m 19s
699:	learn: 28.7079791	total: 10m 4s	remaining: 4m 19s
700:	learn: 28.6982041	total: 10m 5s	remaining: 4m 18s
701:	learn: 2

831:	learn: 27.4580186	total: 12m 29s	remaining: 2m 31s
832:	learn: 27.4508902	total: 12m 30s	remaining: 2m 30s
833:	learn: 27.4456350	total: 12m 31s	remaining: 2m 29s
834:	learn: 27.4404152	total: 12m 32s	remaining: 2m 28s
835:	learn: 27.4282329	total: 12m 34s	remaining: 2m 27s
836:	learn: 27.4153637	total: 12m 35s	remaining: 2m 27s
837:	learn: 27.4006819	total: 12m 36s	remaining: 2m 26s
838:	learn: 27.3899048	total: 12m 37s	remaining: 2m 25s
839:	learn: 27.3786816	total: 12m 37s	remaining: 2m 24s
840:	learn: 27.3735109	total: 12m 38s	remaining: 2m 23s
841:	learn: 27.3655957	total: 12m 40s	remaining: 2m 22s
842:	learn: 27.3559729	total: 12m 41s	remaining: 2m 21s
843:	learn: 27.3501607	total: 12m 41s	remaining: 2m 20s
844:	learn: 27.3413560	total: 12m 42s	remaining: 2m 19s
845:	learn: 27.3316011	total: 12m 43s	remaining: 2m 19s
846:	learn: 27.3244646	total: 12m 44s	remaining: 2m 18s
847:	learn: 27.3177765	total: 12m 45s	remaining: 2m 17s
848:	learn: 27.3131099	total: 12m 46s	remaining:

980:	learn: 26.1834629	total: 15m	remaining: 17.4s
981:	learn: 26.1772364	total: 15m 1s	remaining: 16.5s
982:	learn: 26.1675147	total: 15m 2s	remaining: 15.6s
983:	learn: 26.1620759	total: 15m 3s	remaining: 14.7s
984:	learn: 26.1571541	total: 15m 4s	remaining: 13.8s
985:	learn: 26.1506782	total: 15m 5s	remaining: 12.9s
986:	learn: 26.1414010	total: 15m 6s	remaining: 11.9s
987:	learn: 26.1351487	total: 15m 7s	remaining: 11s
988:	learn: 26.1273515	total: 15m 8s	remaining: 10.1s
989:	learn: 26.1215822	total: 15m 9s	remaining: 9.19s
990:	learn: 26.1120881	total: 15m 10s	remaining: 8.27s
991:	learn: 26.1035849	total: 15m 11s	remaining: 7.35s
992:	learn: 26.0944847	total: 15m 13s	remaining: 6.44s
993:	learn: 26.0876496	total: 15m 13s	remaining: 5.52s
994:	learn: 26.0771884	total: 15m 15s	remaining: 4.6s
995:	learn: 26.0737849	total: 15m 16s	remaining: 3.68s
996:	learn: 26.0673927	total: 15m 17s	remaining: 2.76s
997:	learn: 26.0628981	total: 15m 18s	remaining: 1.84s
998:	learn: 26.0546962	tot

In [8]:
print('Лучшие параметры:' +str(best_xgb_params) + ' , ошибка:' + str(1/(102*739*6.)*Q_xgb))

Лучшие параметры:(0.1, 1000, 10) , ошибка:17.997727734933957


In [9]:
params

[(0.1, 1000, 10)]

In [ ]:
    
%%time
answer = []
xgb_model = CatBoostRegressor(learning_rate=best_xgb_params[0], iterations=best_xgb_params[1], depth=best_xgb_params[2], loss_function='RMSE',
                                    #boosting_type='Plain', loss_function='MultiClass'
            one_hot_max_size=20, )
X_test = dat_pred.copy()
X_test.drop(['zone', 'tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
X_train = dat_train_pred.copy()
X_train.drop(['zone', 'tim1','1_ans','2_ans','3_ans','5_ans','4_ans','6_ans'], inplace=True, axis=1)
Q= 0
for i in range(1,7):
    Q_june = 0. # ошибка в июне
    Y_test = dat_pred[str(i) + '_ans']
    Y_train = dat_train_pred[str(i) + '_ans']
    
    #Считаем ошибку по регрессии
    stud = xgb_model.fit(X_train, Y_train)
    Y_pred = stud.predict(X_test)
    Y_score = stud.score(X_test, Y_test)
    
    Y_pred[Y_pred < 0] = 0  # All low values set to 0
    #models_june[i] = (i, stud)
    dat_pred[str(i) + '_ans_cat'] = Y_pred
    Q_june += sum(abs(Y_test-Y_pred))
    Q+= Q_june
    print ('Tочность модели со значением через часа ' + str(i) + ' = ' + str(Y_score) + ', ошибка= ' +str(1/(102*739*6.)*Q_june))
    for  j in range(len(X_test)):
        answer.append([dat_pred.loc[dat_pred.index[j],'zone'].astype(str)+'_'+dat_pred.loc[dat_pred.index[j],'tim1'].strftime('%Y-%m-%d_%H')+'_'+str(i),int(abs(Y_pred[j]))])
    
    print(' ')
print('Итоговая ошибка за июнь ='+str(1/(102*739*6.)*Q))

0:	learn: 226.2623903	total: 2.22s	remaining: 36m 54s
1:	learn: 205.4394536	total: 4.27s	remaining: 35m 29s
2:	learn: 186.7537801	total: 6.33s	remaining: 35m 3s
3:	learn: 170.2232864	total: 8.43s	remaining: 34m 58s
4:	learn: 155.1080452	total: 10.5s	remaining: 34m 40s
5:	learn: 141.6555958	total: 12.3s	remaining: 33m 54s
6:	learn: 129.6217142	total: 14.2s	remaining: 33m 33s
7:	learn: 118.6883676	total: 16.2s	remaining: 33m 24s
8:	learn: 109.0014197	total: 18s	remaining: 33m 4s
9:	learn: 100.2801705	total: 19.9s	remaining: 32m 50s
10:	learn: 92.6220193	total: 21.7s	remaining: 32m 31s
11:	learn: 85.6956118	total: 23.5s	remaining: 32m 16s
12:	learn: 79.4196076	total: 25.3s	remaining: 32m 2s
13:	learn: 73.9679687	total: 27.2s	remaining: 31m 55s
14:	learn: 69.0529728	total: 29.1s	remaining: 31m 53s
15:	learn: 64.7816210	total: 31.3s	remaining: 32m 7s
16:	learn: 61.0179408	total: 33.3s	remaining: 32m 5s
17:	learn: 57.6463390	total: 35.2s	remaining: 32m 1s
18:	learn: 54.7428725	total: 37.2s	r

150:	learn: 28.8384685	total: 4m 52s	remaining: 27m 23s
151:	learn: 28.8069995	total: 4m 54s	remaining: 27m 20s
152:	learn: 28.7752328	total: 4m 55s	remaining: 27m 17s
153:	learn: 28.7496126	total: 4m 57s	remaining: 27m 15s
154:	learn: 28.7184063	total: 4m 59s	remaining: 27m 14s
155:	learn: 28.6913795	total: 5m 1s	remaining: 27m 13s
156:	learn: 28.6545591	total: 5m 3s	remaining: 27m 11s
157:	learn: 28.6278268	total: 5m 5s	remaining: 27m 10s
158:	learn: 28.5928647	total: 5m 7s	remaining: 27m 7s
159:	learn: 28.5618320	total: 5m 9s	remaining: 27m 5s
160:	learn: 28.5426512	total: 5m 11s	remaining: 27m 3s
161:	learn: 28.5148936	total: 5m 13s	remaining: 27m 1s
162:	learn: 28.4952035	total: 5m 15s	remaining: 26m 57s
163:	learn: 28.4748768	total: 5m 16s	remaining: 26m 55s
164:	learn: 28.4498659	total: 5m 18s	remaining: 26m 52s
165:	learn: 28.4179957	total: 5m 20s	remaining: 26m 50s
166:	learn: 28.3839566	total: 5m 22s	remaining: 26m 49s
167:	learn: 28.3573287	total: 5m 24s	remaining: 26m 47s
1

298:	learn: 25.8715969	total: 9m 14s	remaining: 21m 40s
299:	learn: 25.8571635	total: 9m 16s	remaining: 21m 38s
300:	learn: 25.8409286	total: 9m 18s	remaining: 21m 36s
301:	learn: 25.8227322	total: 9m 19s	remaining: 21m 34s
302:	learn: 25.8095660	total: 9m 21s	remaining: 21m 32s
303:	learn: 25.7974570	total: 9m 23s	remaining: 21m 30s
304:	learn: 25.7838734	total: 9m 25s	remaining: 21m 28s
305:	learn: 25.7713088	total: 9m 27s	remaining: 21m 26s
306:	learn: 25.7585897	total: 9m 28s	remaining: 21m 24s
307:	learn: 25.7472581	total: 9m 30s	remaining: 21m 22s
308:	learn: 25.7343521	total: 9m 32s	remaining: 21m 20s
309:	learn: 25.7180283	total: 9m 34s	remaining: 21m 18s
310:	learn: 25.7080610	total: 9m 36s	remaining: 21m 16s
311:	learn: 25.6907142	total: 9m 38s	remaining: 21m 14s
312:	learn: 25.6733752	total: 9m 39s	remaining: 21m 12s
313:	learn: 25.6619658	total: 9m 41s	remaining: 21m 10s
314:	learn: 25.6512142	total: 9m 43s	remaining: 21m 8s
315:	learn: 25.6401765	total: 9m 44s	remaining: 2

444:	learn: 24.1401804	total: 14m 39s	remaining: 18m 16s
445:	learn: 24.1293653	total: 14m 41s	remaining: 18m 14s
446:	learn: 24.1205138	total: 14m 43s	remaining: 18m 13s
447:	learn: 24.1091964	total: 14m 46s	remaining: 18m 11s
448:	learn: 24.0990345	total: 14m 48s	remaining: 18m 10s
449:	learn: 24.0869769	total: 14m 51s	remaining: 18m 9s
450:	learn: 24.0793743	total: 14m 53s	remaining: 18m 7s
451:	learn: 24.0682603	total: 14m 55s	remaining: 18m 5s
452:	learn: 24.0547222	total: 14m 57s	remaining: 18m 4s
453:	learn: 24.0436504	total: 14m 59s	remaining: 18m 2s
454:	learn: 24.0341237	total: 15m 2s	remaining: 18m
455:	learn: 24.0230929	total: 15m 5s	remaining: 18m
456:	learn: 24.0138594	total: 15m 8s	remaining: 17m 59s
457:	learn: 24.0043426	total: 15m 10s	remaining: 17m 57s
458:	learn: 23.9905861	total: 15m 13s	remaining: 17m 56s
459:	learn: 23.9811277	total: 15m 15s	remaining: 17m 54s
460:	learn: 23.9727260	total: 15m 18s	remaining: 17m 53s
461:	learn: 23.9630313	total: 15m 20s	remaining

589:	learn: 22.8044634	total: 20m 26s	remaining: 14m 12s
590:	learn: 22.7956872	total: 20m 28s	remaining: 14m 10s
591:	learn: 22.7896632	total: 20m 31s	remaining: 14m 8s
592:	learn: 22.7830400	total: 20m 33s	remaining: 14m 6s
593:	learn: 22.7771341	total: 20m 35s	remaining: 14m 4s
594:	learn: 22.7711923	total: 20m 38s	remaining: 14m 2s
595:	learn: 22.7617129	total: 20m 40s	remaining: 14m
596:	learn: 22.7538363	total: 20m 42s	remaining: 13m 58s
597:	learn: 22.7482546	total: 20m 44s	remaining: 13m 56s
598:	learn: 22.7397075	total: 20m 46s	remaining: 13m 54s
599:	learn: 22.7345064	total: 20m 48s	remaining: 13m 52s
600:	learn: 22.7279067	total: 20m 51s	remaining: 13m 50s
601:	learn: 22.7153282	total: 20m 53s	remaining: 13m 48s
602:	learn: 22.7047871	total: 20m 55s	remaining: 13m 46s
603:	learn: 22.6969961	total: 20m 57s	remaining: 13m 44s
604:	learn: 22.6878347	total: 20m 59s	remaining: 13m 42s
605:	learn: 22.6811896	total: 21m 2s	remaining: 13m 40s
606:	learn: 22.6733845	total: 21m 4s	rem

735:	learn: 21.7170686	total: 26m 37s	remaining: 9m 32s
736:	learn: 21.7115165	total: 26m 39s	remaining: 9m 30s
737:	learn: 21.7016736	total: 26m 40s	remaining: 9m 28s
738:	learn: 21.6971481	total: 26m 42s	remaining: 9m 26s
739:	learn: 21.6895689	total: 26m 44s	remaining: 9m 23s
740:	learn: 21.6829234	total: 26m 46s	remaining: 9m 21s
741:	learn: 21.6747210	total: 26m 48s	remaining: 9m 19s
742:	learn: 21.6695092	total: 26m 50s	remaining: 9m 17s
743:	learn: 21.6652885	total: 26m 52s	remaining: 9m 14s
744:	learn: 21.6594182	total: 26m 54s	remaining: 9m 12s
745:	learn: 21.6544686	total: 26m 56s	remaining: 9m 10s
746:	learn: 21.6480713	total: 26m 58s	remaining: 9m 8s
747:	learn: 21.6421149	total: 27m	remaining: 9m 5s
748:	learn: 21.6330181	total: 27m 2s	remaining: 9m 3s
749:	learn: 21.6290736	total: 27m 3s	remaining: 9m 1s
750:	learn: 21.6243530	total: 27m 5s	remaining: 8m 59s
751:	learn: 21.6184330	total: 27m 7s	remaining: 8m 56s
752:	learn: 21.6143499	total: 27m 9s	remaining: 8m 54s
753:	

883:	learn: 20.8194293	total: 31m 31s	remaining: 4m 8s
884:	learn: 20.8143628	total: 31m 33s	remaining: 4m 6s
885:	learn: 20.8088838	total: 31m 36s	remaining: 4m 3s
886:	learn: 20.8056871	total: 31m 38s	remaining: 4m 1s
887:	learn: 20.8026025	total: 31m 40s	remaining: 3m 59s
888:	learn: 20.7959207	total: 31m 42s	remaining: 3m 57s
889:	learn: 20.7888080	total: 31m 44s	remaining: 3m 55s
890:	learn: 20.7814697	total: 31m 46s	remaining: 3m 53s
891:	learn: 20.7773236	total: 31m 48s	remaining: 3m 51s
892:	learn: 20.7705509	total: 31m 50s	remaining: 3m 48s
893:	learn: 20.7660826	total: 31m 52s	remaining: 3m 46s
894:	learn: 20.7620963	total: 31m 54s	remaining: 3m 44s
895:	learn: 20.7547166	total: 31m 57s	remaining: 3m 42s
896:	learn: 20.7512748	total: 31m 59s	remaining: 3m 40s
897:	learn: 20.7461741	total: 32m 1s	remaining: 3m 38s
898:	learn: 20.7414850	total: 32m 3s	remaining: 3m 36s
899:	learn: 20.7337549	total: 32m 5s	remaining: 3m 33s
900:	learn: 20.7289080	total: 32m 7s	remaining: 3m 31s


In [ ]:
answer_1 = pd.DataFrame(answer)

In [ ]:
answer_1.columns = ['id','y']

In [ ]:
answer_1.to_csv('c6w6-ans-catboost-0.csv', sep=',', index=False)

In [ ]:
print('Итоговая  ошибка за июнь ='+str(1/(102*739*6.)*Q))

In [ ]:
!pip install shap
import ipywidgets as widgets
widgets.Widget.widget_types.values()